In [1]:
!pip install pyresparser fitz PyMuPDF pypdf2 >/dev/null

In [3]:
!ln -sf "NOUVELLE FLORE DU LIBAN ET DE LA SYRIE.pdf" document.pdf

In [4]:
import fitz
document = 'document.pdf'
filename = document
search_term = "the"
pdf_document = fitz.open(filename)

In [5]:
doc = pdf_document
dir(doc)

['FontInfos',
 'FormFonts',
 'Graftmaps',
 'PDFCatalog',
 'PDFTrailer',
 'ShownPages',
 '__class__',
 '__contains__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 '_addFormFont',
 '_delToC',
 '_delXmlMetadata',
 '_deleteObject',
 '_deletePage',
 '_do_links',
 '_dropOutline',
 '_embeddedFileAdd',
 '_embeddedFileDel',
 '_embeddedFileGet',
 '_embeddedFileIndex',
 '_embeddedFileInfo',
 '_embeddedFileNames',
 '_embeddedFileUpd',
 '_forget_page',
 '_getCharWidths',
 '_getMetadata',
 '_getNewXref',
 '_getOLRootNumber',
 '_getPDFfileid',
 '_getPDFroot',
 '_getPageInfo',
 '_getPageObjNumber',
 '_getPag

In [6]:
doc.getToC()

[[1, 'Abreviations', 77]]

In [7]:
doc.searchPageFor(528, 'Pteranthus')

[Rect(37.68000030517578, 99.17700958251953, 113.67733764648438, 113.85101318359375),
 Rect(37.91999816894531, 164.3159942626953, 94.519287109375, 179.55099487304688)]

In [8]:
# https://pymupdf.readthedocs.io/en/latest/faq/#how-to-analyze-font-characteristics

import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)


doc = fitz.open("document.pdf")
page = doc[528]

In [9]:
#header = 'Family	Genus	epithet	author	syn.	Plate	description	Floraison from	Floraison to	Floraison notes	L.	S.	Aire géogr.	Notes'
header = 'Family	Genus	epithet	author	syn.	Plate	description	Floraison	L.	S.	Aire géogr.	Notes'
columns = header.split('	')
columns

['Family',
 'Genus',
 'epithet',
 'author',
 'syn.',
 'Plate',
 'description',
 'Floraison',
 'L.',
 'S.',
 'Aire géogr.',
 'Notes']

In [10]:
class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)
print(Face("Times",10.929, 0, 4))

def isfamily(t, fp):
    font = "Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000"
    if re.match('^[A-Z]+$', t) or str(font) == fp:
        #print(f'''Text: {t}\n{fp}\n\n''')
        return True
data = {}
item = dict.fromkeys(columns)
print(item)
p = (doc[x] for x in range(516,604))
p = (doc[x] for x in range(527,604))
p

Font: 'Times' (serifed, proportional), size 10.929, color #000000
{'Family': None, 'Genus': None, 'epithet': None, 'author': None, 'syn.': None, 'Plate': None, 'description': None, 'Floraison': None, 'L.': None, 'S.': None, 'Aire géogr.': None, 'Notes': None}


<generator object <genexpr> at 0x7fb3ae5961d0>

In [13]:
#aaron was in the middle of working on this and it has an error we know about
# import re

# def floraison(doc, pagenum=0, block_start=0):
#     for page in next(p):
#         blocks = page.getText("dict", flags=11)["blocks"]
#         for x,b in enumerate(blocks[block_start:]):  # iterate through the text blocks
#             for y,l in enumerate(b["lines"]):  # iterate through the text lines
#                 for z,s in enumerate(l["spans"]):  # iterate through the text spans
#                     s["style"] = flags_decomposer(s["flags"])  # readable font flags
#                     face = Face(s["font"], s["size"], s["color"], s["flags"])
#                     if re.match("Floraison", s["text"]):
#                         yield (page.number, x, y, z, s["text"])

# p = (doc[x] for x in range(516,540))
# next(floraison(doc, x))

In [14]:
p = (doc[x] for x in range(527,604))
p

<generator object <genexpr> at 0x7fb3ae2fd650>

In [15]:
page.number

528

In [16]:
page = next(p)
# read page text as a dictionary, suppressing extra spaces in CJK fonts
blocks = page.getText("dict", flags=11)["blocks"]
for x,b in enumerate(blocks):  # iterate through the text blocks
    for y,l in enumerate(b["lines"]):  # iterate through the text lines
        for z,s in enumerate(l["spans"]):  # iterate through the text spans
            s["style"] = flags_decomposer(s["flags"])  # readable font flags
            face = Face(s["font"], s["size"], s["color"], s["flags"])
            if isfamily(s["text"], face) and len(l["spans"]) == 1:
                item['Family'] = s["text"]
                print(x, len(blocks), y, z, s["text"], face)
                break
            if not item['Family']:
                break
            if s['font'] == 'Times-Bold':
                item['Genus'], *item['epithet'] = s["text"].split(' ')
                item['epithet'] = item['epithet'][0] if item['epithet'] else l["spans"][1]["text"]
                currsize = s['size']
#                print(x, len(blocks), y, z, s["text"], font_properties)
                print(x, len(blocks), y, len(b["lines"]), z, s["text"], face)
                nn, xx, yy, zz, tt = next(floraison(page, x))
                item['Floraison'] = tt
#                 for q in range(x, max(x+15, len(blocks))):
#                     for ll in blocks[q]["lines"]:
#                         s = ll["spans"][0]
#                         text, *rest = s['text'].split(' ')
#                         if text == "Floraison:":
#                             face = Face(s["font"], s["size"], s["color"], s["flags"])
#                             print(face, s['text'])
#                             for ss in ll["spans"]:
#                                 #print(ss['text'], end=" ")
#                                 pass
                from pprint import pprint
                print(x, len(blocks), y, z, pprint(item))
                #print([a["text"] for a in l["spans"]])
                print(); print(); 
                break


15 18 0 0 CARYOPHYLLACEAE Font: 'Times-Roman' (serifed, proportional), size 10.9449, color #000000


In [17]:
search = ((x, doc.searchPageFor(x, "Floraison", 100, False)) for x in range(516,520))

In [18]:
next(search)

(516,
 [Rect(42.0, 69.48302459716797, 77.89947509765625, 81.48902130126953),
  Rect(42.0, 324.6029968261719, 78.0272445678711, 336.6089782714844)])

In [19]:
p = (doc[x] for x in range(516,520))

In [20]:
page = next(p)
rects = page.searchFor("Floraison")
rects[0]

Rect(42.0, 69.48302459716797, 77.89947509765625, 81.48902130126953)

In [21]:
page.getTextBlocks()

[(177.83999633789062,
  18.363027572631836,
  409.36810302734375,
  30.36902618408203,
  'NOUVELLE FLORE  439',
  0,
  0),
 (24.0,
  40.73700714111328,
  409.758544921875,
  67.1710205078125,
  'centes. Sépales linéaires, spatules, denticulés, lacérés. Staminodes plus courts que\nleur partie soudée. Fruit un peu accrescent, à la fin glabre.',
  1,
  0),
 (42.0,
  69.48302459716797,
  409.81085205078125,
  94.92902374267578,
  'Floraison: automne.\nS. St. Entre Alep et Bagdad (Oliv), Palmyre (Mt, Zoh, Pb), Deir-ez-Zor (Gb), entre Mey-',
  2,
  0),
 (23.760000228881836,
  92.52300262451172,
  409.95843505859375,
  114.12897491455078,
  'yadine et Salihiyé (Hd-Mz), W. de Deir-ez-Zor, entre Abou Kemal et Meyyadine (Pb), entre\nSoukhné et Deir-ez-Zor (Dlb).',
  3,
  0),
 (41.52000045776367,
  116.0429916381836,
  154.27919006347656,
  128.0489959716797,
  'Aire géogr. — Syrie et Iraq.',
  4,
  0),
 (45.84000015258789,
  134.33700561523438,
  409.8203430175781,
  149.01101684570312,
  'HALOG

In [22]:
d = iter(doc)

In [23]:
next(d).getTextBlocks()

[(110.87999725341797,
  124.28401184082031,
  344.0527648925781,
  140.29200744628906,
  'AUTRES OUVRAGES DU MÊME AUTEUR',
  0,
  0),
 (59.040000915527344,
  183.02999877929688,
  419.97821044921875,
  196.3699951171875,
  'La végétation arborescente des Pays du Levant, dans : Publications techniques et',
  1,
  0),
 (35.52000045776367,
  195.89700317382812,
  370.8548889160156,
  210.57101440429688,
  "scientifiques de l'École Française d'Ingénieurs de Beyrouth, n° 13, 1947.",
  2,
  0),
 (59.279998779296875,
  218.07000732421875,
  420.3532409667969,
  231.41000366210938,
  "Contribution à l'étude de la flore syrienne et libanaise. Série de courts mémoires",
  3,
  0),
 (35.52000045776367,
  231.177001953125,
  408.53289794921875,
  245.85101318359375,
  'insérés dans : Bulletin de la Société Botanique de France, depuis 1952, tome 98.',
  4,
  0),
 (59.279998779296875,
  252.5369873046875,
  420.21917724609375,
  267.21099853515625,
  'La Flore du Djebel Druze. Université Saint-Josep

In [24]:
doc[621].getTextBlocks()

[(47.040000915527344,
  32.29714584350586,
  278.7561340332031,
  47.56802749633789,
  '546\nNOUVELLE FLORE',
  0,
  0),
 (58.79999923706055,
  59.93598556518555,
  143.0931396484375,
  290.4889831542969,
  'gussonii Pari.\nhordeaceus L.\nintermedius Guss.\njaponicus Thunb.\nlanceolatus Roth\nmacrpstachys Desf.\nmadritensis L.\nmaximus Desf.\nmollis L.\noxyphlaeus Payne\npalaestinus Melderis\npatulus Mert. et Koch\npinnatus L.\nramosus L.\nrigens L.\nrigidus Roth\nrubens L.\nscoparius L.\nsquarrosus L.\nsterilis L.\nsyriacus Boiss. et Bl.\ntectorum L.\ntomentellus Boiss.\nvillosus Forsk.',
  1,
  0),
 (45.119998931884766,
  287.843017578125,
  174.9159393310547,
  310.16900634765625,
  "Broussonetia papyrifera (L.) L'Hér.\nBuffonia",
  2,
  0),
 (58.31999969482422,
  307.0429992675781,
  130.1332550048828,
  376.8479919433594,
  'ephedrina Sam.\nkotschyana Boiss.\nmacrospermaj. Gay\nmulticeps Decne\npaniculata Dubois\ntenuifolia Vill.\nvirgata Boiss.',
  3,
  0),
 (44.400001525878906,


In [25]:
doc[621].getText()

"546\nNOUVELLE FLORE\ngussonii Pari.\nhordeaceus L.\nintermedius Guss.\njaponicus Thunb.\nlanceolatus Roth\nmacrpstachys Desf.\nmadritensis L.\nmaximus Desf.\nmollis L.\noxyphlaeus Payne\npalaestinus Melderis\npatulus Mert. et Koch\npinnatus L.\nramosus L.\nrigens L.\nrigidus Roth\nrubens L.\nscoparius L.\nsquarrosus L.\nsterilis L.\nsyriacus Boiss. et Bl.\ntectorum L.\ntomentellus Boiss.\nvillosus Forsk.\nBroussonetia papyrifera (L.) L'Hér.\nBuffonia\nephedrina Sam.\nkotschyana Boiss.\nmacrospermaj. Gay\nmulticeps Decne\npaniculata Dubois\ntenuifolia Vill.\nvirgata Boiss.\nBulbillaria gageoides Zucc.\nBUTOMACEAE\nButomus\numbellatus L.\nC\nCalamagrostis\nepigeios (L.) Roth\nlittorea (Schrad.) D.C.\npseudophragmites (Hall Fil.) Kœl.\nCalligonum\ncomosum L'Hér.\ntetrapterum Jaub. et Spach\nCannabis\nsativa L.\nCarex\nacuta L.\nacutiformis Ehrh.\nbanatica Heuff.\nbuekii Wimm.\ncontigua Hoppe\ndistans L.\ndiversicolor Druce\ndivisa Huds.\ndivulsa Stokes\n123\n129\n120, 128\n120, 129\n119,

In [26]:
page.getFontList()

[(48515, 'n/a', 'Type1', 'Times-Roman', 'F0', 'WinAnsiEncoding'),
 (48516, 'n/a', 'Type1', 'Times-Italic', 'F1', 'WinAnsiEncoding'),
 (48584, 'pfa', 'Type1', 'OYLUZG+Times-Roman', 'F12', 'WinAnsiEncoding'),
 (48575, 'pfa', 'Type1', 'RCBBCE+Times-Italic', 'F13', 'WinAnsiEncoding'),
 (48535, 'n/a', 'Type1', 'Times-Bold', 'F2', 'WinAnsiEncoding'),
 (48526, 'pfa', 'Type1', 'QKXTUK+Times-Roman', 'F24', 'WinAnsiEncoding'),
 (48517, 'pfa', 'Type1', 'KLTUIO+Times-Italic', 'F25', 'WinAnsiEncoding'),
 (48566, 'pfa', 'Type1', 'XTFERS+Times-Bold', 'F26', 'WinAnsiEncoding'),
 (48603, 'pfa', 'Type1', 'TCNEMC+Times-BoldItalic', 'F27', 'WinAnsiEncoding'),
 (48537, 'pfa', 'Type1', 'ETVWNV+Times-Roman', 'F28', 'WinAnsiEncoding'),
 (48546, 'pfa', 'Type1', 'CPDYZV+Times-Italic', 'F29', 'WinAnsiEncoding'),
 (48565, 'n/a', 'Type1', 'Times-BoldItalic', 'F3', 'WinAnsiEncoding'),
 (48556, 'pfa', 'Type1', 'VHVHMH+Times-Bold', 'F30', 'WinAnsiEncoding'),
 (48593, 'n/a', 'Type1', 'Helvetica', 'F4', 'WinAnsiEncodin